In [1]:
from pytube import YouTube, Playlist
import cv2
import numpy as np
import re
from matplotlib import pyplot as plt
from IPython.display import clear_output
import os
from youtube_transcript_api.formatters import JSONFormatter
import tqdm
#import moviepy.editor as mpe
import pytube
from youtube_transcript_api import YouTubeTranscriptApi

# Folder creation

In [2]:
root_path = "Matignon-LSFv1_data"
os.makedirs(root_path, exist_ok=True)
os.makedirs(root_path+'/video', exist_ok=True)
os.makedirs(root_path+'/audio', exist_ok=True)
os.makedirs(root_path+'/subtitles', exist_ok=True)
os.makedirs(root_path+'/cropped_videos', exist_ok=True)

# Video, subtitle and audio downloading
Set the YouTube Playlist

In [3]:
#p = Playlist('https://www.youtube.com/playlist?list=PLQo4IzmQ0lgnIRQe0tmWr9qCn7_vQNXWy')
p = Playlist('https://www.youtube.com/watch?v=XanQoeiLfSI&list=PLSa7CvPe2Ias7ME-30VzqBoT_Ae1YT6Ph')

In [4]:
prefix = ""
i = 0
total_duration_in_s = 0
for video in tqdm.tqdm(p.videos):
    # Download subtitles
    try:
        video_id = video.video_id
        subtitles = YouTubeTranscriptApi.get_transcript(video.video_id, languages=['fr', 'fr-FR'])
        transcript_list = YouTubeTranscriptApi.list_transcripts(video.video_id)
        for transcript in transcript_list:
            if not transcript.is_generated:
                subtitles = transcript.fetch()
                break
        formatter = JSONFormatter()

        # .format_transcript(transcript) turns the transcript into a JSON string.
        json_formatted = formatter.format_transcript(subtitles)

        # Now we can write it out to a file.
        with open(root_path+'/subtitles/'+video.video_id+'.json', "w") as json_file:
            json_file.write(json_formatted)


        vname = root_path+'/video/'+video.video_id+"_clip.mp4"
        aname = root_path+'/audio/'+video.video_id+"_audio.mp3"

        # Download video and rename
        stream_to_download = video.streams.filter(
            subtype='mp4', res="1080p").first()
        video_file = stream_to_download.download()
        os.rename(video_file, vname)

        # Download audio and rename
        audio = video.streams.filter(only_audio=True).first().download()
        os.rename(audio, aname)

        


        i+=1
    except  Exception as e:
        print(e, video.video_id)
        continue

 44%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                                                                                                                                                                                            | 7/16 [01:00<01:04,  7.19s/it]


Could not retrieve a transcript for the video https://www.youtube.com/watch?v=qfRpUR1aR0s! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem! qfRpUR1aR0s


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [02:22<00:00,  8.91s/it]


# Cropping the videos

In [5]:
def crop_video(filename, out_filename, fps, x0, x1, y0, y1, visualize=False):

    cap = cv2.VideoCapture(filename)
    out = cv2.VideoWriter(out_filename, cv2.VideoWriter_fourcc(*'MP4V'), fps, (x1-x0, y1-y0))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    i =0
    while cap.isOpened():
        ret, frame = cap.read()
        if ret != True:
            break
        i+=1
        if visualize:
            if (i%16 == 0):
                clear_output(wait=True)
                plt.axis("off")
                plt.imshow(frame[:, :, ::-1])
                plt.show()
                plt.imshow(frame[y0:y1, x0:x1, ::-1])
                plt.show()
        out.write(frame[y0:y1, x0:x1])
        
    out.release()
    return frame_count
    

It might be necessary to adjust the bounding box position for the cropping

In [ ]:
for file in os.listdir(root_path+'/video/'):
    outfilename = (root_path+'/cropped_videos/'+file).replace(".mp4", "_cropped.mp4")
    print(outfilename)
    crop_video(root_path+'/video/'+file, outfilename,  30, 1334, 1334+494, 417, 417+494, visualize=False)


OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


Matignon-LSFv1_data/cropped_videos/XanQoeiLfSI_clip_cropped.mp4


In [ ]:
# Kept as a history of bounding boxes positions
#crop_video(root_path+'/video/Not_cropped/'+i, 30, 1380, 1380+494, 381, 381+494)
#crop_video(root_path+'/video/new/'+i,(root_path+'/video/Cropped/'+i).replace(".mp4", "_cropped.mp4")  30, 1335, 1335+494, 381, 381+494)
#crop_video(root_path+'/video/Not_cropped/'+i, 30, 1358, 1358+494, 381, 381+494)
#crop_video(root_path+'/video/Not_cropped/'+i, 30, 1527, 1527+345, 520, 520+345)